In [ ]:
# import libraries
import hashlib
import datetime
import json

In [ ]:
# Block class
class Block:
  def __init__(self, timestamp, transactions, previous_hash):
    self.timestamp = timestamp
    self.transactions = transactions
    self.previous_hash = previous_hash
    self.hash = self.generate_hash()

  def generate_hash(self):
    timestamp_str = str(self.timestamp).encode('utf-8')
    previous_hash_str = str(self.previous_hash).encode('utf-8')
    transactions_str = json.dumps(self.transactions, sort_keys=True).encode('utf-8')
    combined_bytes = timestamp_str + previous_hash_str + transactions_str
    return hashlib.sha256(combined_bytes).hexdigest()

In [ ]:
# BlockChain class
class BlockChain:
  def __init__(self):
    self.chain = [self.create_first_block()]

  def create_first_block(self):
    return Block(datetime.datetime.now, "", "0")

  def get_latest_block(self):
    return self.chain[-1]

  def create_block(self, new_block):
    new_block.previous_hash = self.get_latest_block().hash
    new_block.hash = new_block.generate_hash()
    self.chain.append(new_block)

  def add_transaction(self, sender, receiver, amount):
      transaction = {
          'sender': sender,
          'receiver': receiver,
          'amount': amount
      }
      self.create_block(Block(datetime.datetime.now, transaction, self.get_latest_block().hash))

  def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            if current_block.hash != current_block.generate_hash():
                return 'Not valid'

            if current_block.previous_hash != previous_block.hash:
                return 'Not valid'
        return 'Valid'

In [ ]:
# Test
test_blockchain = BlockChain()

test_blockchain.add_transaction('Ali', 'Zakat Organization', 1000)
test_blockchain.add_transaction('Abu', 'Zakat Organization', 1200)

In [ ]:
# Print the BlockChain
for block in test_blockchain.chain:
  print("Timestamp:", block.timestamp)
  print("Transactions:", block.transactions)
  print("Previous Hash:", block.previous_hash)
  print("Hash:", block.hash)
  print("\n")

Timestamp: <built-in method now of type object at 0xa32ba0>
Transactions: 
Previous Hash: 0
Hash: 5da5e97ce9167924bd74313f1dc57b5031137caef7a8c99f86d8a6388ac3fc3a


Timestamp: <built-in method now of type object at 0xa32ba0>
Transactions: {'sender': 'Ali', 'receiver': 'Zakat Organization', 'amount': 1000}
Previous Hash: 5da5e97ce9167924bd74313f1dc57b5031137caef7a8c99f86d8a6388ac3fc3a
Hash: 14f1cc5ae78c8571d580dc98788a2c095716ffe6c3d757e747a7c612fe72882d


Timestamp: <built-in method now of type object at 0xa32ba0>
Transactions: {'sender': 'Abu', 'receiver': 'Zakat Organization', 'amount': 1200}
Previous Hash: 14f1cc5ae78c8571d580dc98788a2c095716ffe6c3d757e747a7c612fe72882d
Hash: 8b838d937f386563d3c609e791c507ef495170269986bd66e4c9eefb8e71d8c3




In [ ]:
# Check BlockChain validation
print("Is blockchain valid?")
test_blockchain.is_chain_valid()

Is blockchain valid?


'Valid'